In [ ]:
import sys
sys.path.append('/home/janvanrijn/projects/openml-python')
sys.path.append('/home/janvanrijn/projects/openml-python-contrib')

import arff
import openmlcontrib
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import theano.tensor as tt

In [ ]:
x_columns = ['svc__gamma', 'svc__C']
y_column = 'predictive_accuracy'

with open('/home/janvanrijn/projects/openml-multitask/data/svm-sklearn-bot.arff', 'r') as fp:
    dataset = arff.load(fp)
dataset = openmlcontrib.meta.arff_to_dataframe(dataset)
dataset = dataset[dataset['task_id'] == 6]

X = np.array(dataset[x_columns].values, dtype=np.float)
y = np.array(dataset[y_column].values, dtype=np.float)
train_size = 200


X_tr = X[:train_size]
X_te = X[train_size:]

y_tr = y[:train_size]
y_te = y[train_size:]


In [ ]:
with pm.Model() as marginal_gp_model:
    # Specify the covariance function.
    l1 = pm.Uniform('l1', 0, 10)
    l2 = pm.Uniform('l2', 0, 10)
    cov_func = pm.gp.cov.ExpQuad(2, ls=[l1, l2])

    # Specify the GP.  The default mean function is `Zero`.
    gp = pm.gp.Marginal(cov_func=cov_func)

    # The scale of the white noise term can be provided,
    sigma = pm.HalfCauchy("sigma", beta=5)
    y_ = gp.marginal_likelihood("y", X=X_tr, y=y_tr, noise=sigma)
    trace = pm.sample(50, progressbar=True)

In [ ]:
mu, var = gp.predict(X_te)
print(mu.shape)